In [8]:
# Import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, fbeta_score

**1. Wczytanie danych** <br>
Wybrane do analizy dane dotyczą rodzajów wina oraz ich parametrów fizykochemicznych takich jak np. zawartość magnezu, kolor czy zawartość fenoli.

In [9]:
url = "https://raw.githubusercontent.com/zuzannabrauer/machineLearning/master/lab10/wine.data"
names = ['Wine type', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
wine_data = pd.read_csv(url, sep = ',', names = names)
wine_data

,Wine type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


**2. Podział zbioru danych na zbiór testowy i treningowy** <br>
Dane zostały podzielone na dwa zbiory: treningowy oraz testowy w stosunku odpowiednio 80% i 20%. Warto dodać, że w tym przypadku zmiennymi, których klasyfikatory będą używać do kategoryzacji danych są parametry fizykochemiczne wina a klasę stanowi typ napoju.

In [10]:
X = wine_data.drop('Wine type', axis=1)
y = wine_data['Wine type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**3. Normalizacja danych** <br>
Zbiór testowy oraz treningowy został poddany normalizacji ponieważ, algorytm KNN, którego będziemy używać do późniejszej klasyfikacji jest wrażliwy na magnitudę danych i może dawać złe wyniki. Po drugie znormalizowanie danych przyśpiesza proces optymalizacji oraz sprawia, że algorytmy ML nie przykładają zbyt dużej uwagi do cech, które miały początkowo największe wartości.

In [11]:
scaler = preprocessing.MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

X_train.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0.715054,0.513238,0.634409,0.484536,0.402174,0.262590,0.027523,0.698113,0.120253,0.392491,0.358974,0.201465,0.310263
1,0.446237,0.160896,0.483871,0.520619,0.108696,0.100719,0.302752,0.849057,0.379747,0.151024,0.358974,0.289377,0.163087
2,0.158602,0.254582,0.494624,0.381443,0.304348,0.690647,0.547401,0.075472,0.294304,0.168089,0.529915,0.619048,0.043755
3,0.637097,0.645621,0.602151,0.639175,0.347826,0.251799,0.085627,0.566038,0.313291,0.513652,0.136752,0.106227,0.365951
4,0.663978,0.217923,0.672043,0.484536,0.282609,0.514388,0.654434,0.283019,0.392405,0.191126,0.495726,0.934066,0.441527


**4. Trening oraz predykcja na zbiorze testowym używając algorytmów KNN i RF**

In [12]:
neigh = KNeighborsClassifier(n_neighbors = 6)
knn = neigh.fit(X_train, y_train)

clf = RandomForestClassifier(max_depth = 2, random_state = 42)
rfc = clf.fit(X_train, y_train)

In [13]:
y_pred_rfc = rfc.predict(X_test)
y_pred_knn = knn.predict(X_test)

**6. Zapoznanie się metrykami do oceny modeli dostępnymi w scikit learn**
* acuuracy_score - podaje wartość procentową lub liczbową (normalize = False) prawidłowych predykcji
* precision_score - zwraca tp/(tp + fp), gdzie tp są to dane zaklasyfikowane jako prawdziwie dodatnie a fp zaklasyfikowane jako fałszywie dodatnie. Pozwala to określić zdolność klasyfikatora do nieklasyfikowania próbek fp jako tp.
* recall_score - zwraca tp/(tp + fn), gdzie tp są to dane zaklasyfikowane jako prawdziwie dodatnie a fp zaklasyfikowane jako fałszywie negatywne. Im jego wartość jest bliższa 1 tym lepiej.
* F-measures:
  * f1_score - oblicza wartość f1, która jest średnią harmoniczną pomiędzy precision a recall
  * fbeta_score - oblicza wartość fbeta, która jest średnią ważoną pomiędzy precision a recall
* confusion_matrix - zwraca macierz zawierającą liczbę poprawnych i błędnych klasyfikacji dla kazdej z klas
* classification_ report - raport zawierający wartości precision, f1-score, support dla każdej z klas oraz średnie wartości dla accuracy

**7. Metryki dla modelu KNN i RF** <br>

**A. metryki dla KNN**

In [16]:
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Precision:", precision_score(y_test, y_pred_knn, average = 'weighted'))
print("Recall:", recall_score(y_test, y_pred_knn, average = 'weighted'))
print("F1 Score:", f1_score(y_test, y_pred_knn, average = 'weighted'))
print("F-beta Score ", fbeta_score(y_test, y_pred_knn, beta = 0.5, average = 'weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))
print("Classification Report:\n", classification_report(y_test, y_pred_knn))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
F-beta Score  1.0
Confusion Matrix:
 [[14  0  0]
 [ 0 16  0]
 [ 0  0  6]]
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00         6

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



**B. metryki dla RF**

In [17]:
print("Accuracy:", accuracy_score(y_test, y_pred_rfc))
print("Precision:", precision_score(y_test, y_pred_rfc, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_rfc, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_rfc, average='weighted'))
print("F-beta Score ", fbeta_score(y_test, y_pred_rfc, beta=0.5, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rfc))
print("Classification Report:\n", classification_report(y_test, y_pred_rfc))

Accuracy: 0.9444444444444444
Precision: 0.9484126984126983
Recall: 0.9444444444444444
F1 Score: 0.9449984172206395
F-beta Score  0.9467221231927115
Confusion Matrix:
 [[13  1  0]
 [ 0 15  1]
 [ 0  0  6]]
Classification Report:
               precision    recall  f1-score   support

           1       1.00      0.93      0.96        14
           2       0.94      0.94      0.94        16
           3       0.86      1.00      0.92         6

    accuracy                           0.94        36
   macro avg       0.93      0.96      0.94        36
weighted avg       0.95      0.94      0.94        36



**8. Wnioski** <br>
 Spoglądając na powyższe wyniki można wyciągnąć pochopny wniosek, że model KNN jest lepszy, ponieważ we wszystkich metrykach osiągnął on wartość 100% i nie popełnił żadnego błędu w klasyfikacji (confussion matrix). Jest to jednak sytuacja zbyt idealistyczna i się nie zdarzająca się w rzeczywistości. Mamy tutaj prawdopodbnie doczynienia z overffitingiem dla algorytmu K najbliższych sąsiadów. Może wynikać to na przykład ze zbyt małej liczby danych uczących. <br> Biorąc pod uwagę powyższe argumenty model Random Forest wydaje się lepszy. Nie ma on idelanych wyników, ale na pewno nie jest przeuczony, a wartości metryk na poziomie 95% są i tak bardzo wysokie.
